In [1]:
import pandas as pd
import glob
import numpy as np
import datetime
import re
import os

In [2]:
file_list = glob.glob('raw_data/*')
df = pd.read_csv(file_list[0])
df['Brand'] = os.path.splitext(os.path.basename(file_list[0]))[0].split('_')[0]
df['Name'] = os.path.splitext(os.path.basename(file_list[0]))[0]

for f in file_list[1:]:
    temp_df = pd.read_csv(f)
    temp_df['Brand'] = os.path.splitext(os.path.basename(f))[0].split('_')[0]
    temp_df['Name'] = os.path.splitext(os.path.basename(f))[0]
    
    df = pd.concat([df, temp_df])
df

,Price,作業系統與版本,處理器品牌,處理器型號,處理器核心數,ROM儲存空間,主螢幕尺寸,主螢幕解析度,主螢幕像素密度,主螢幕最大亮度,...,第二前相機光圈F,實用工具,第三前相機光圈F,第三前相機感光元件,時間顯示,計步器,第四主相機等效焦距,內建觸控筆,吊飾孔,雙卡雙通
0,"$20,900",iOS 13,Apple,A13 Bionic,6.0,256 GB,6.1 inch,1792x828 pixels,324 ppi,650 nits,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"$23,300",iOS 13,Apple,A13 Bionic,6.0,512 GB,5.8 inch,2436x1125 pixels,463 ppi,1200 nits,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"$26,000",iOS 14,Apple,A14 Bionic,NaN,256 GB,6.1 inch,2532x1170 pixels,457 ppi,1200 nits,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"$22,300",iOS 14,Apple,A14 Bionic,NaN,256 GB,5.4 inch,2340x1080 pixels,477 ppi,1200 nits,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"$31,590",iOS 14,Apple,A14 Bionic,NaN,512 GB,6.1 inch,2532x1170 pixels,457 ppi,1200 nits,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,"$5,690",Android 11,MediaTek,Dimensity 700,8.0,128 GB,6.5 inch,2400x1080 pixels,405 ppi,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"$6,850",Android 11,Qualcomm,Snapdragon 732G,8.0,128 GB,6.67 inch,2400x1080 pixels,395 ppi,1200 nits,...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN
0,"$4,600",Android 10,MediaTek,Helio G85,8.0,128 GB,6.53 inch,2340x1080 pixels,395 ppi,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"$5,190",Android 10,MediaTek,Dimensity 800U,8.0,128 GB,6.53 inch,2340x1080 pixels,395 ppi,450 nits,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.shape

(208, 123)

## SOC

In [4]:
df = df.dropna(subset=["處理器型號"])

In [5]:
soc_list = df['處理器型號'].unique()

In [6]:
soc_score = pd.read_csv('soc_score.csv').set_index('Unnamed: 0').T.to_dict('list')

In [7]:
for s in soc_list:
    s_split = str(s).split()
    if s_split[-1] == 'Bionic':
        s = 'Apple '+s.split()[0]
    elif 'MediaTek'  in s_split and 'Dimensity' in s_split:
        s = s.split()[-2] + ' '+s.split()[-1]
    elif s == 'MT6771' or s == 'MT6762V':
        s = 'Helio P60'
    elif s == 'MT6762D':
        s = 'Helio P22'
    if s not in soc_score:
        if s[:1] == 'T':
            s = 'Unisoc '+s
            if s not in soc_score:
                print(s)   
        else:
            print(s)

In [8]:
def soc2score(x):
    s = x['處理器型號']
    s_split = str(s).split()
    if s_split[-1] == 'Bionic':
        s = 'Apple '+s.split()[0]
    elif 'MediaTek'  in s_split and 'Dimensity' in s_split:
        s = s.split()[-2] + ' '+s.split()[-1]
    elif s == 'MT6771' or s == 'MT6762V':
        s = 'Helio P60'
    elif s == 'MT6762D':
        s = 'Helio P22'
    if s not in soc_score:
        if s[:1] == 'T':
            s = 'Unisoc '+s
    x['soc_s'] = soc_score[s][0]
    x['soc_m'] = soc_score[s][1]
    
    return x
    
df = df.apply(soc2score, axis=1)

In [9]:
df.columns

Index(['Price', '作業系統與版本', '處理器品牌', '處理器型號', '處理器核心數', 'ROM儲存空間', '主螢幕尺寸',
       '主螢幕解析度', '主螢幕像素密度', '主螢幕最大亮度',
       ...
       '第三前相機光圈F', '第三前相機感光元件', '時間顯示', '計步器', '第四主相機等效焦距', '內建觸控筆', '吊飾孔',
       '雙卡雙通', 'soc_s', 'soc_m'],
      dtype='object', length=125)

In [10]:
df = df.drop(['處理器型號', '處理器品牌', '處理器核心數'], axis=1)

In [11]:
df = df.drop(['處理器時脈','圖形處理器'], axis=1)

In [12]:
df.columns

Index(['Price', '作業系統與版本', 'ROM儲存空間', '主螢幕尺寸', '主螢幕解析度', '主螢幕像素密度', '主螢幕最大亮度',
       '主螢幕材質', '主螢幕觸控', '主螢幕更新率',
       ...
       '第三前相機光圈F', '第三前相機感光元件', '時間顯示', '計步器', '第四主相機等效焦距', '內建觸控筆', '吊飾孔',
       '雙卡雙通', 'soc_s', 'soc_m'],
      dtype='object', length=120)

## Price

In [13]:
df['Price'] = df['Price'].apply(lambda x: int(x.replace('$', '').replace(',','')))

## ROM

In [14]:
df = df.drop(['作業系統與版本'], axis=1)

In [15]:
df = df.dropna(subset=["ROM儲存空間"])

In [16]:
def handle_rom(x):
    if 'TB' in x:
        return 1024*(int(x.replace('TB', '')))
    else:
        return int(x.replace('GB', ''))
    
df['ROM儲存空間'] = df['ROM儲存空間'].apply(handle_rom)

## Screen

In [17]:
# df[df['主螢幕尺寸'].str.contains('inch') != True]
df['主螢幕尺寸'] = df['主螢幕尺寸'].apply(lambda x: float(str(x).replace('inch', '')))

In [18]:
df = df.dropna(subset=["主螢幕解析度"])

In [19]:
pattern_resolution = '[\d]+x[\d]+'

def handle_pix(x):
    reso = re.search(pattern_resolution, x['主螢幕解析度']).group().split('x')
    x['resolution_x'] = reso[0]
    x['resolution_y'] = reso[1]
    return x
    
df = df.apply(handle_pix, axis=1)

In [20]:
df = df.drop(['主螢幕解析度'], axis=1)

In [21]:
df['主螢幕像素密度'] = df['主螢幕像素密度'].apply(lambda x: int(x.replace('ppi', '')))

In [22]:
df = df.drop(['主螢幕像素密度'], axis=1)

In [23]:
df = df.drop(['主螢幕最大亮度', '主螢幕觸控'], axis=1)

In [24]:
df['OLED'] = df['主螢幕材質'].str.contains('OLED') == True

In [25]:
df = df.drop(['主螢幕材質'], axis=1)

In [26]:
df['90Hz'] = df['主螢幕更新率'].str.contains('90') == True
df['120Hz'] = df['主螢幕更新率'].str.contains('120') == True
df['144Hz'] = df['主螢幕更新率'].str.contains('144') == True

In [27]:
df = df.drop(['主螢幕更新率'], axis=1)

In [28]:
df = df.drop(['主螢幕觸控採樣率'], axis=1)

## Comm.

In [29]:
df = df.drop(['4G FDD LTE', '4G TDD LTE', '3G頻率', '2G頻率', '4G LTE', 'VoLTE',
       'SIM Card類型', 'Wi-Fi'], axis=1)

In [30]:
df['Wifi5G'] = df['IEEE 802.11傳輸速度'].str.contains('5GH')
df['Wifi6'] = df['IEEE 802.11傳輸速度'].str.contains('6')
df = df.drop(['IEEE 802.11傳輸速度'], axis=1)

In [31]:
df['NFC'] = df['NFC'] == 'Yes'

In [32]:
df = df.drop(['藍牙', '衛星定位'], axis=1)

In [33]:
df['5G連網'] = (df['5G連網'] == 'Yes')

In [34]:
df = df.drop(['5G NR頻率', '5G NR頻段', '5G連網', 'SIM1'], axis=1)

In [35]:
df = df.drop(['SIM卡槽數', 'SIM卡槽設計', 'SIM2', '5G組網方式'], axis=1)

In [36]:
df = df.drop(['CA支援頻段組合', 'LTE Cat.', 'CA載波聚合','SIM卡槽或記憶卡擇一'], axis=1)

In [37]:
df = df.drop(['雙卡雙通'], axis=1)

In [38]:
df['藍牙版本'].value_counts()

5.0    94
5.1    54
5.2    24
4.2    20
4.0     2
4.1     1
Name: 藍牙版本, dtype: int64

In [39]:
df['bt5.0'] = (df['藍牙版本'] == 5.0)
df['bt5.1'] = (df['藍牙版本'] == 5.1)
df['bt5.2'] = (df['藍牙版本'] == 5.2)
df['bt4.2'] = (df['藍牙版本'] == 4.2)

In [40]:
df = df.drop(['藍牙版本'], axis=1)

In [41]:
df.columns[20:]

Index(['NFC', '快充功率', '無線充電', '陀螺儀', '加速度感應器', '機身材質', '機身長度', '機身寬度', '機身厚度',
       '機身重量', '防水防塵等級', '傳輸埠', '麥克風', '機身顏色', 'Brand', 'Name', '第三主相機畫素',
       '第三主相機感光元件', '第三主相機光圈F', '主螢幕耐用性', '前相機自動對焦', '高度氣壓感測器', '第三主相機等效焦距',
       '指紋辨識器', '最大通話時間', 'RAM記憶體', '電池容量', '主相機8K錄影', '快充技術', '裝置分類', '記憶體格式',
       '儲存空間格式', '前相機等效焦距', '3.5mm耳機孔', '副螢幕材質', '記憶卡', '最大擴充儲存空間', '最大待機時間',
       '主螢幕佔比', 'RAW檔拍攝', '副螢幕尺寸', '可替換電池', '無線反向充電', '第四主相機畫素', '第四主相機感光元件',
       '第四主相機光圈F', '喇叭', '紅外線', '副螢幕解析度', '影片播放格式', '音樂播放器', '圖片支援格式', '副螢幕觸控',
       '錄影格式', '第二前相機畫素', '第二前相機感光元件', '第二前相機光圈F', '實用工具', '第三前相機光圈F',
       '第三前相機感光元件', '時間顯示', '計步器', '第四主相機等效焦距', '內建觸控筆', '吊飾孔', 'soc_s',
       'soc_m', 'resolution_x', 'resolution_y', 'OLED', '90Hz', '120Hz',
       '144Hz', 'Wifi5G', 'Wifi6', 'bt5.0', 'bt5.1', 'bt5.2', 'bt4.2'],
      dtype='object')

## RAM

In [42]:
df[df['RAM記憶體'].isnull()]['Name']

0            apple_iphone_11
0        apple_iphone_11_pro
0            apple_iphone_12
0       apple_iphone_12_mini
0        apple_iphone_12_pro
0    apple_iphone_12_pro_max
0            apple_iphone_13
0       apple_iphone_13_mini
0        apple_iphone_13_pro
0    apple_iphone_13_pro_max
0       apple_iphone_se_2020
0            apple_iphone_xr
0               mto_m68_plus
Name: Name, dtype: object

In [43]:
name2ram = {'mto_m68_plus': 0.5,
            'apple_iphone_xr': 3,
            'apple_iphone_se_2020': 3,
            'apple_iphone_11': 4,
            'apple_iphone_11_pro': 4,
            'apple_iphone_12': 4,
            'apple_iphone_12_mini': 4,
            'apple_iphone_12_pro': 6,
            'apple_iphone_12_pro_max': 6,
            'apple_iphone_13': 4,
            'apple_iphone_13_mini': 4,
            'apple_iphone_13_pro': 6,
            'apple_iphone_13_pro_max': 6
           }
def handle_ram(x):
    ram = str(x['RAM記憶體'])
    
    if 'B' not in ram:
        return name2ram[x['Name']]
    else:
        if 'GB' in ram:
            return float(ram.replace('GB', ''))
        elif 'MB' in ram:
            return float(ram.replace('MB', ''))/1024
        
df['RAM記憶體'] = df.apply(handle_ram, axis=1)

## Other about memory

In [44]:
df['UFS'] = df['儲存空間格式'].str.contains('UFS') == True

In [45]:
df['記憶卡'] = ~df['記憶卡'].isnull()

In [46]:
df = df.drop(['最大擴充儲存空間','記憶體格式','儲存空間格式'], axis=1)

## Camera

In [47]:
df.columns

Index(['Price', 'ROM儲存空間', '主螢幕尺寸', '主相機畫素', '主相機感光元件', '主相機光圈F', '主相機等效焦距',
       '主相機LED補光燈', '主相機自動對焦', '主相機光學防手震', '主相機UHD 4K錄影', '第二主相機畫素',
       '第二主相機感光元件', '第二主相機光圈F', '第二主相機等效焦距', '前相機畫素', '前相機感光元件', '前相機光圈F',
       '前相機LED補光燈', '前相機UHD 4K錄影', 'NFC', '快充功率', '無線充電', '陀螺儀', '加速度感應器',
       '機身材質', '機身長度', '機身寬度', '機身厚度', '機身重量', '防水防塵等級', '傳輸埠', '麥克風', '機身顏色',
       'Brand', 'Name', '第三主相機畫素', '第三主相機感光元件', '第三主相機光圈F', '主螢幕耐用性',
       '前相機自動對焦', '高度氣壓感測器', '第三主相機等效焦距', '指紋辨識器', '最大通話時間', 'RAM記憶體', '電池容量',
       '主相機8K錄影', '快充技術', '裝置分類', '前相機等效焦距', '3.5mm耳機孔', '副螢幕材質', '記憶卡',
       '最大待機時間', '主螢幕佔比', 'RAW檔拍攝', '副螢幕尺寸', '可替換電池', '無線反向充電', '第四主相機畫素',
       '第四主相機感光元件', '第四主相機光圈F', '喇叭', '紅外線', '副螢幕解析度', '影片播放格式', '音樂播放器',
       '圖片支援格式', '副螢幕觸控', '錄影格式', '第二前相機畫素', '第二前相機感光元件', '第二前相機光圈F', '實用工具',
       '第三前相機光圈F', '第三前相機感光元件', '時間顯示', '計步器', '第四主相機等效焦距', '內建觸控筆', '吊飾孔',
       'soc_s', 'soc_m', 'resolution_x', 'resolution_y', 'OLED', '90Hz',
       '120Hz', '144Hz', 'W

In [48]:
df = df.drop(['主相機感光元件', '主相機光圈F', '主相機等效焦距', '主相機LED補光燈', '主相機自動對焦',
              '第二主相機感光元件', '第二主相機光圈F', '第二主相機等效焦距',
             '第三主相機感光元件', '第三主相機光圈F','第三主相機等效焦距',
             '第四主相機感光元件', '第四主相機光圈F','第四主相機等效焦距',
              '前相機感光元件', '前相機光圈F', '前相機LED補光燈','前相機自動對焦',
              '第二前相機感光元件', '第二前相機光圈F'
             ], axis=1)

In [49]:
df = df.drop(['前相機等效焦距', '第三前相機光圈F', '第三前相機感光元件'], axis=1)

In [50]:
df.columns

Index(['Price', 'ROM儲存空間', '主螢幕尺寸', '主相機畫素', '主相機光學防手震', '主相機UHD 4K錄影',
       '第二主相機畫素', '前相機畫素', '前相機UHD 4K錄影', 'NFC', '快充功率', '無線充電', '陀螺儀',
       '加速度感應器', '機身材質', '機身長度', '機身寬度', '機身厚度', '機身重量', '防水防塵等級', '傳輸埠',
       '麥克風', '機身顏色', 'Brand', 'Name', '第三主相機畫素', '主螢幕耐用性', '高度氣壓感測器', '指紋辨識器',
       '最大通話時間', 'RAM記憶體', '電池容量', '主相機8K錄影', '快充技術', '裝置分類', '3.5mm耳機孔',
       '副螢幕材質', '記憶卡', '最大待機時間', '主螢幕佔比', 'RAW檔拍攝', '副螢幕尺寸', '可替換電池', '無線反向充電',
       '第四主相機畫素', '喇叭', '紅外線', '副螢幕解析度', '影片播放格式', '音樂播放器', '圖片支援格式', '副螢幕觸控',
       '錄影格式', '第二前相機畫素', '實用工具', '時間顯示', '計步器', '內建觸控筆', '吊飾孔', 'soc_s',
       'soc_m', 'resolution_x', 'resolution_y', 'OLED', '90Hz', '120Hz',
       '144Hz', 'Wifi5G', 'Wifi6', 'bt5.0', 'bt5.1', 'bt5.2', 'bt4.2', 'UFS'],
      dtype='object')

In [51]:
df['主相機光學防手震'] = df['主相機光學防手震'] == 'Yes'
df['主相機UHD 4K錄影'] = df['主相機UHD 4K錄影'] == 'Yes'
df['前相機UHD 4K錄影'] = df['前相機UHD 4K錄影'] == 'Yes'
df['主相機8K錄影'] = df['主相機8K錄影'] == 'Yes'

In [52]:
def to_MP(x):
    x = str(x)
    if 'n' in x:
        return 0
    elif '萬畫素' in x:
        return float(x.replace('萬畫素', ''))/100
    elif '億畫素' in x:
        return float(x.replace('億畫素', ''))*100
    else:
        return float(x)/100
        
        
df['主相機畫素'] = df['主相機畫素'].apply(to_MP)
df['第二主相機畫素'] = df['第二主相機畫素'].apply(to_MP)
df['第三主相機畫素'] = df['第三主相機畫素'].apply(to_MP)
df['第四主相機畫素'] = df['第四主相機畫素'].apply(to_MP)
df['前相機畫素'] = df['前相機畫素'].apply(to_MP)
df['第二前相機畫素'] = df['第二前相機畫素'].apply(to_MP)

In [53]:
def calculate_cam_num(x):
    if x['第四主相機畫素'] != 0:
        return 4
    elif x['第三主相機畫素'] != 0:
        return 3
    elif x['第二主相機畫素'] != 0:
        return 2
    elif x['主相機畫素'] != 0:
        return 1
    else:
        return 0
df['cam_back_num'] = df.apply(calculate_cam_num, axis=1)

In [54]:
df = df.drop(['錄影格式'], axis=1)

In [55]:
df['RAW檔拍攝'] = df['RAW檔拍攝'] == 'Yes'

## Battery

In [56]:
dict_charging = {
    'asus_zenfone_6_zs630kl': 18,
    'asus_zenfone_7_pro_zs671ks': 30,
    'asus_zenfone_7_zs670ks': 30,
    'blackshark_2': 27,
    'cat_s61': 18,
    'htc_desire_20plus': 18,
    'htc_desire_20_pro': 18,
    'huawei_mate_20_x_5g': 40,
    'lg_v60_thinq': 25,
    'lg_velvet': 27,
    'lg_wing': 25,
    'motorola_one_hyper': 45,
    'oppo_a91': 20,
    'sony_xperia_1_ii': 18,
    'sugar_t50': 15
}

def handle_charging(x):
    charging = str(x['快充功率'])
    if 'n' in charging and str(x['快充技術']) != 'nan':
        return dict_charging[x['Name']]
    elif 'n' in charging and x['Brand'] == 'apple':
        return 18
    elif 'n' in charging:
        return 10
    else:
        return float(charging.replace('W', ''))
        

df['快充功率'] = df.apply(handle_charging, axis=1)

In [57]:
df['無線充電'] = df['無線充電'] == 'Yes'

In [58]:
df['可替換電池'] = df['可替換電池'] == 'Yes'
df['無線反向充電'] = df['無線反向充電'] == 'Yes'

In [59]:
df = df.drop(['最大通話時間','快充技術'], axis=1)

In [60]:
df = df.drop(['最大待機時間'], axis=1)

In [61]:
df[df['電池容量'].isnull()]['Name']

0            apple_iphone_11
0        apple_iphone_11_pro
0            apple_iphone_12
0       apple_iphone_12_mini
0        apple_iphone_12_pro
0    apple_iphone_12_pro_max
0            apple_iphone_13
0       apple_iphone_13_mini
0        apple_iphone_13_pro
0    apple_iphone_13_pro_max
0       apple_iphone_se_2020
0            apple_iphone_xr
Name: Name, dtype: object

In [62]:
dict_battery = {
    'apple_iphone_11': 3110,
    'apple_iphone_11_pro': 3046,
    'apple_iphone_12': 2815,
    'apple_iphone_12_mini': 2227,
    'apple_iphone_12_pro':  2815,
    'apple_iphone_12_pro_max': 3687,
    'apple_iphone_13': 3095,
    'apple_iphone_13_mini' : 2406,
    'apple_iphone_13_pro': 3095,
    'apple_iphone_13_pro_max': 4352,
    'apple_iphone_se_2020': 1821,
    'apple_iphone_xr': 2658
}

def handle_battery(x):
    b = str(x['電池容量'])
    if 'mAh' in b:
        return int(b.replace('mAh', ''))
    elif 'nan' in b:
        return dict_battery[x['Name']]
    else:
        return int(b)
df['電池容量'] = df.apply(handle_battery, axis=1)

## 外觀

In [63]:
df['glass'] = df['機身材質'].str.contains('玻璃') == True
df['plastic'] = df['機身材質'].str.contains('塑料') == True
df = df.drop(['機身材質'], axis=1)

In [64]:
df['摺疊螢幕'] = (df['裝置分類'].str.contains('摺疊螢幕') | df['裝置分類'].str.contains('疊螢幕'))

In [65]:
df = df.drop(['裝置分類'], axis=1)

In [66]:
def handle_whl(x):
    x['機身長度'] =  float(str(x['機身長度']).replace('mm', ''))
    x['機身寬度'] =  float(str(x['機身寬度']).replace('mm', ''))
    x['機身厚度'] =  float(str(x['機身厚度']).replace('mm', ''))
    
    return x

df = df.apply(handle_whl, axis=1)

In [67]:
df['副螢幕觸控'] = ((~df['副螢幕觸控'].isnull()) & df['副螢幕觸控'].str.contains('Yes'))

In [68]:
def handle_weight(x):
    if x['Name'] == 'koobee_s16':
        return 157
    else:
        return float(str(x['機身重量']).replace('g', ''))
df['機身重量'] = df.apply(handle_weight, axis=1)

In [69]:
df['機身顏色'] = df['機身顏色'].apply(lambda x: len(x.split(', ')))


## Sensor

In [70]:
df.columns

Index(['Price', 'ROM儲存空間', '主螢幕尺寸', '主相機畫素', '主相機光學防手震', '主相機UHD 4K錄影',
       '第二主相機畫素', '前相機畫素', '前相機UHD 4K錄影', 'NFC', '快充功率', '無線充電', '陀螺儀',
       '加速度感應器', '機身長度', '機身寬度', '機身厚度', '機身重量', '防水防塵等級', '傳輸埠', '麥克風',
       '機身顏色', 'Brand', 'Name', '第三主相機畫素', '主螢幕耐用性', '高度氣壓感測器', '指紋辨識器',
       'RAM記憶體', '電池容量', '主相機8K錄影', '3.5mm耳機孔', '副螢幕材質', '記憶卡', '主螢幕佔比',
       'RAW檔拍攝', '副螢幕尺寸', '可替換電池', '無線反向充電', '第四主相機畫素', '喇叭', '紅外線', '副螢幕解析度',
       '影片播放格式', '音樂播放器', '圖片支援格式', '副螢幕觸控', '第二前相機畫素', '實用工具', '時間顯示', '計步器',
       '內建觸控筆', '吊飾孔', 'soc_s', 'soc_m', 'resolution_x', 'resolution_y',
       'OLED', '90Hz', '120Hz', '144Hz', 'Wifi5G', 'Wifi6', 'bt5.0', 'bt5.1',
       'bt5.2', 'bt4.2', 'UFS', 'cam_back_num', 'glass', 'plastic', '摺疊螢幕'],
      dtype='object')

In [71]:
df = df.drop(['陀螺儀', '加速度感應器', '指紋辨識器'], axis=1)

## Other

In [72]:
df = df.drop(['喇叭', '紅外線','影片播放格式',
       '音樂播放器', '圖片支援格式',
             '實用工具', '時間顯示', '計步器', '內建觸控筆',
       '吊飾孔'], axis=1)

In [73]:
df['防水防塵等級'] = ~df['防水防塵等級'].isnull()

In [74]:
df['USB3'] = ((~df['傳輸埠'].isnull()) & df['傳輸埠'].str.contains('3'))
df = df.drop(['傳輸埠'], axis=1)

In [75]:
df['3.5mm耳機孔'] = ((~df['3.5mm耳機孔'].isnull()) & df['3.5mm耳機孔'].str.contains('Yes'))

In [76]:
df = df.drop(['麥克風'], axis=1)

In [77]:
df = df.drop(['副螢幕尺寸', '高度氣壓感測器', '副螢幕材質'], axis=1)

In [78]:
df = df.drop(['主螢幕佔比', '主螢幕耐用性'], axis=1)

In [79]:
df.to_csv('sogi_df.csv', index=False)

In [80]:
df = pd.read_csv('sogi_df.csv')
df

,Price,ROM儲存空間,主螢幕尺寸,主相機畫素,主相機光學防手震,主相機UHD 4K錄影,第二主相機畫素,前相機畫素,前相機UHD 4K錄影,NFC,...,bt5.0,bt5.1,bt5.2,bt4.2,UFS,cam_back_num,glass,plastic,摺疊螢幕,USB3
0,20900,256,6.10,12.0,True,True,12.0,12.0,True,True,...,True,False,False,False,False,2,True,False,False,False
1,23300,512,5.80,12.0,True,True,12.0,12.0,True,True,...,True,False,False,False,False,3,True,False,False,False
2,26000,256,6.10,12.0,True,True,12.0,12.0,True,True,...,True,False,False,False,False,2,True,False,False,False
3,22300,256,5.40,12.0,True,True,12.0,12.0,True,True,...,True,False,False,False,False,2,True,False,False,False
4,31590,512,6.10,12.0,True,True,12.0,12.0,True,True,...,True,False,False,False,False,3,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,5690,128,6.50,48.0,False,False,2.0,8.0,False,True,...,False,True,False,False,True,3,False,False,False,False
197,6850,128,6.67,108.0,False,True,8.0,16.0,False,True,...,False,True,False,False,True,4,False,False,False,False
198,4600,128,6.53,48.0,False,True,8.0,13.0,False,True,...,True,False,False,False,False,4,False,False,False,False
199,5190,128,6.53,48.0,False,True,2.0,13.0,False,True,...,False,True,False,False,True,3,False,True,False,False


In [81]:
df.drop(['Name','副螢幕解析度'], axis=1).to_csv('sogi_mobile.csv', index=False)

In [82]:
set(df.columns)-set(['Price', 'ROM儲存空間', '主螢幕尺寸', '主相機畫素', '主相機光學防手震', '主相機UHD 4K錄影',
       '第二主相機畫素', '前相機畫素', '前相機UHD 4K錄影', 'NFC', '快充功率', '無線充電', 
        'Brand', '防水防塵等級','機身長度','機身重量',
       '機身寬度', '機身厚度','副螢幕觸控',
       '第三主相機畫素','RAM記憶體', '電池容量', '主相機8K錄影', 
       '記憶卡', 'RAW檔拍攝',  '可替換電池','USB3','3.5mm耳機孔',
       '無線反向充電', '第四主相機畫素',  '第二前相機畫素', 'soc_s', 'soc_m',
       'resolution_x', 'resolution_y', 'OLED', '90Hz', '120Hz', '144Hz',
       'Wifi5G', 'Wifi6', 'bt5.0', 'bt5.1', 'bt5.2', 'bt4.2', 'UFS','摺疊螢幕',
       'cam_back_num', 'glass', 'plastic'])

{'Name', '副螢幕解析度', '機身顏色'}